## SLC Flavour Customization

Since in the default SLC of Exasol, there is no 'XGBoost' module installed and the tutorial for SLC also won't work becuase it is too large to upload on the SaaS platform as of now. We need to create out own own

This notebook will guide you just how to do that.

# Configure Tutorial Script Languages Container

## Prerequisites

Prior to using this notebook the following steps need to be completed:
1. [Configure the AI-Lab](../main_config.ipynb).

### Open Secure Configuration Storage

In [11]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

Output()

Box(children=(Box(children=(Label(value='Configuration Store', layout=Layout(border_bottom='solid 1px', border…

### Import some utility functions

In [20]:
%run ./utils/file_system_ui.ipynb
%run ./utils/slc_ui.ipynb

Output()



### Instantiate SLCT Manager

The "Script-Languages-Container-Tools" Manager (SLCT Manager) simplifies using the API of `exaslct`.
The following cell will therefore create an instance class `SlctManager` from the notebook-connector.

In [21]:
from exasol.nb_connector import slct_manager
slctmanager = slct_manager.SlctManager(ai_lab_config)

## Customize

First you need to define an alias for the new SLC. The alias will be used to reference the container later from the UDFs.

Note: In this tutorial the alias also will be used as part of the export file (tar.gz) and the uploaded container to the BucketFS. This allows you to create, upload and use different script-language-containers: one per alias.

In [23]:
display(get_alias_ui(ai_lab_config, "ai_lab_default_custom"))

Box(children=(Box(children=(Label(value='Language Alias', layout=Layout(border_bottom='solid 1px', border_left…

### Flavor Customization Build Step

`exasclt` consists of multiple build steps. By a build step here we mean a file structure which serves as an input for a certain stage of the building process of the script-languages-container. See [Advanced Topics](./advanced.ipynb) for more details.

Build step **flavor_customization** is defined by a Dockerfile and several package lists. We recommend to add new packages to the package lists and only modify the Dockerfile if you need very specific changes, like adding additional resources.

In [25]:
show_directory_content(slctmanager.slc_dir.flavor_dir / "flavor_customization")

/home/jupyter/notebooks/script_languages_container/script_languages_release/flavors/template-Exasol-all-python-3.10/flavor_customization/Dockerfile

We are now going to append Python package "xgboost" to one of the package lists by adding `xgboost|2.0.3` to file `flavor_customization/packages/python3_pip_packages`. 
Notes:
 - running the following command multiple times will iteratively append the packages
 - you can also click on the link and modify the file directly

In [26]:
xgboost_pkg = slct_manager.PipPackageDefinition(pkg="xgboost", version="2.0.3")
slctmanager.append_custom_packages([xgboost_pkg])
show_files([slctmanager.slc_dir.custom_pip_file])

/home/jupyter/notebooks/script_languages_container/script_languages_release/flavors/template-Exasol-all-python-3.10/flavor_customization/packages/python3_pip_packages

#### Rebuilding the customized Flavor

After changing the flavor you need to rebuild it. You can do it by running `export` again. Exaslct automatically recognizes that the flavor has changed and builds a new version of the container. Don't get confused by the warnings: `exaslct` first tries to find the cached docker images (see [Advanced Topics](../script_languages_container/advanced.ipynb)), but as the content has changed, the cached image is not available, and the docker service returns a 404 error message.

In [27]:
slctmanager.export()

WARNING - AnalyzeFlavorCustomization_a97a7a05ac(job_id=2025_10_06_14_20_12_1_ExportContainers, no_cache=False, flavor_path=flavors/template-Exasol-all-python-3.10): Image exasol/script-language-container:template-Exasol-all-python-3.10-flavor_customization_PUHAWGLYDJ5CYVVFYDHUNCW7YMVO2T6FQQXIFDBCFYVKZ3CM5TQQ not in registry, got exception 404 Client Error for http+docker://localhost/v1.51/images/create?tag=template-Exasol-all-python-3.10-flavor_customization_PUHAWGLYDJ5CYVVFYDHUNCW7YMVO2T6FQQXIFDBCFYVKZ3CM5TQQ&fromImage=exasol%2Fscript-language-container: Not Found ("manifest for exasol/script-language-container:template-Exasol-all-python-3.10-flavor_customization_PUHAWGLYDJ5CYVVFYDHUNCW7YMVO2T6FQQXIFDBCFYVKZ3CM5TQQ not found: manifest unknown: manifest unknown")
WARNING - AnalyzeRelease_a97a7a05ac(job_id=2025_10_06_14_20_12_1_ExportContainers, no_cache=False, flavor_path=flavors/template-Exasol-all-python-3.10): Image exasol/script-language-container:template-Exasol-all-python-3.10-re

#### Find the Path of our custom SLC
Now we need to find the path of our custom SLC

In [28]:
show_directory_content(slctmanager.working_path.export_path)

/home/jupyter/notebooks/sales_forecasting/container/template-Exasol-all-python-3.10_release_ai_lab_default.tar.gz.sha512sum

/home/jupyter/notebooks/sales_forecasting/container/template-Exasol-all-python-3.10_release_ai_lab_default.tar.gz

#### Upload the Container to BucketFS

Copy the path (.tar.gz file) and open it as a binary

In [30]:
import pickle
from exasol.nb_connector.connections import open_bucketfs_connection
from stopwatch import Stopwatch

with open('container/template-Exasol-all-python-3.10_release_ai_lab_default.tar.gz','rb') as f:
    bucket = open_bucketfs_connection(ai_lab_config)
    bucket.upload('test_slc.tar.gz', f)

# Serialize the model into a byte-array and upload it to the BucketFS, 
# where it will be saved in the file with the specified name.



#### Check the path of the container in the Bucket

In [31]:
from exasol.nb_connector.connections import open_bucketfs_connection
bucket = open_bucketfs_connection(ai_lab_config)
print(bucket.udf_path)

/buckets/bfsdefault/default


In [32]:
print(ai_lab_config.slc_target_dir)

/home/jupyter/notebooks/script_languages_container/script_languages_release


Go to the [main](./main.ipynb#Create-a-command-for-SLC-uploading) file now and to create a querry 